
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_2932615/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 19:43:03.359673: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:43:03.392750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 19:43:03.392789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 19:43:03.393830: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 19:43:03.399736: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:43:03.400279: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[40,20]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.02])

Testando combinacao1: Hidden Size=[40, 20], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.0929750539999077
Stopped at epoch:  13
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.3304237150005065
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.4593304160007392
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.30499600200164423
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.33486546099993575
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.3142472409999755
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 7 ++++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_1_0,0.817267,0.460702,0.611931,0.638233,0.202232,0.114081,0.671072,0.376194
model_1_1_2,0.831188,0.009272,0.516926,0.515269,0.186825,0.209574,0.835362,0.504062
model_1_1_1,0.831199,0.430101,0.621513,0.642435,0.186813,0.120554,0.654502,0.371824
model_1_1_4,0.838739,-0.668047,0.543278,0.462946,0.178468,0.352852,0.789791,0.558471
model_1_1_3,0.858192,0.104173,0.541690,0.544868,0.156940,0.189500,0.792538,0.473282
model_1_1_5,0.868621,-1.077872,0.711593,0.550527,0.145398,0.439544,0.498731,0.467398
model_1_1_6,0.873009,-1.054148,0.725744,0.564157,0.140542,0.434526,0.474260,0.453224
model_1_1_7,0.904219,-0.427727,0.767391,0.664211,0.106001,0.302015,0.402241,0.349180
model_1_1_19,0.904226,-0.427739,0.767424,0.664236,0.105993,0.302018,0.402185,0.349154
model_1_1_18,0.904226,-0.427739,0.767424,0.664236,0.105993,0.302018,0.402185,0.349154


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.3211994839984982
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.35512608099998033
Stopped at epoch:  10
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.23516621599992504
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.28096578899931046
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.20192261900047015
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.20122184100000595
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.215415

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_2_10,0.809024,-4.083741,-7.600571,-3.361581,0.211354,0.773796,0.541997,0.664711
model_1_2_9,0.827530,-3.522547,-6.621993,-2.874433,0.190874,0.688376,0.480329,0.590469
model_1_2_8,0.847916,-2.919627,-5.478430,-2.333131,0.168312,0.596606,0.408263,0.507974
model_1_2_7,0.867544,-2.349140,-4.282651,-1.798832,0.146590,0.509772,0.332906,0.426546
model_1_2_6,0.885936,-1.782867,-3.147919,-1.278557,0.126236,0.423580,0.261397,0.347255
model_1_2_5,0.900372,-1.274846,-2.269105,-0.838959,0.110259,0.346254,0.206015,0.280260
model_1_2_0,0.904226,0.517493,0.633565,0.673571,0.105993,0.073442,0.023092,0.049748
model_1_2_4,0.912604,-0.763392,-1.504302,-0.419702,0.096722,0.268406,0.157818,0.216365
model_1_2_3,0.923730,-0.241430,-0.798704,-0.006419,0.084409,0.188958,0.113352,0.153380
model_1_2_1,0.928805,0.379943,0.506306,0.576079,0.078792,0.094379,0.031112,0.064606


DataFrame salvo em ../results/metrics_1_2
+++++++++++ [1_3] | 1 ++++++++++++++++++
Elapsed time:  0.27168065999831015
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_3] | 2 ++++++++++++++++++
Elapsed time:  0.4645220940001309
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_3] | 3 ++++++++++++++++++
Elapsed time:  0.34370496700103104
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_3] | 4 ++++++++++++++++++
Elapsed time:  0.30800835599984566
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_3] | 5 ++++++++++++++++++
Elapsed time:  0.218049819000953
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_3] | 6 ++++++++++++++++++
Elapsed time:  0.3212585940000281
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 7 ++++++++++++++++++
Elapsed time:  0.2956533060

...

DataFrame salvo em ../results/metrics_4_7
+++++++++++ [4_8] | 1 ++++++++++++++++++
Elapsed time:  0.47788401099933253
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_8] | 2 ++++++++++++++++++
Elapsed time:  0.23250805299903732
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [4_8] | 3 ++++++++++++++++++
Elapsed time:  0.3271236910004518
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_8] | 4 ++++++++++++++++++
Elapsed time:  0.2778321749992756
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_8] | 5 ++++++++++++++++++
Elapsed time:  0.31723416299973906
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [4_8] | 6 ++++++++++++++++++
Elapsed time:  0.32192793899957906
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [4_8] | 7 ++++++++++++++++++
Elapsed time:  0.29788635

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_8_6,0.859817,0.553527,0.986650,0.760373,0.155141,0.411827,0.012629,0.223969
model_4_8_5,0.859881,0.577219,0.988675,0.773716,0.155070,0.389973,0.010713,0.211498
model_4_8_7,0.860175,0.534660,0.984122,0.749312,0.154745,0.429229,0.015021,0.234308
model_4_8_8,0.860558,0.517221,0.981309,0.738861,0.154321,0.445315,0.017682,0.244076
model_4_8_9,0.860900,0.500560,0.978414,0.728777,0.153943,0.460684,0.020420,0.253501
model_4_8_4,0.861248,0.612992,0.990069,0.793070,0.153558,0.356977,0.009395,0.193409
model_4_8_10,0.861289,0.485218,0.975504,0.719375,0.153512,0.474835,0.023173,0.262288
model_4_8_11,0.861809,0.471626,0.972736,0.710956,0.152937,0.487372,0.025791,0.270157
model_4_8_12,0.862557,0.460284,0.970219,0.703831,0.152109,0.497834,0.028172,0.276817
model_4_8_3,0.863520,0.658140,0.990199,0.816721,0.151043,0.315331,0.009271,0.171303


DataFrame salvo em ../results/metrics_4_8
+++++++++++ [4_9] | 1 ++++++++++++++++++
Elapsed time:  0.3819556170001306
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [4_9] | 2 ++++++++++++++++++
Elapsed time:  0.3739279979999992
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [4_9] | 3 ++++++++++++++++++
Elapsed time:  0.27905557400117686
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [4_9] | 4 ++++++++++++++++++
Elapsed time:  0.3512083329987945
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_9] | 5 ++++++++++++++++++
Elapsed time:  0.31641041900002165
Stopped at epoch:  6
1/1 [==============================] - 0s 33ms/step
+++++++++++ [4_9] | 6 ++++++++++++++++++
Elapsed time:  0.3667752469991683
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_9] | 7 ++++++++++++++++++
Elapsed time:  0.3101650040

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_9_4,0.868823,0.778397,0.307800,0.736500,0.145174,0.367435,0.127286,0.254423
model_4_9_3,0.868874,0.788156,0.322238,0.746666,0.145118,0.351254,0.124631,0.244608
model_4_9_5,0.869417,0.771759,0.292840,0.729124,0.144517,0.378442,0.130037,0.261545
model_4_9_2,0.869501,0.800552,0.338831,0.759423,0.144424,0.330699,0.121580,0.232290
model_4_9_1,0.870467,0.814242,0.361084,0.773863,0.143355,0.308001,0.117488,0.218348
model_4_9_6,0.870607,0.768178,0.276221,0.724381,0.143200,0.384378,0.133093,0.266126
model_4_9_0,0.871443,0.827561,0.392644,0.788800,0.142274,0.285917,0.111684,0.203925
model_4_9_7,0.872257,0.767127,0.257879,0.721781,0.141374,0.386121,0.136466,0.268636
model_4_9_8,0.874207,0.767885,0.238232,0.720709,0.139216,0.384865,0.140078,0.269671
model_4_9_9,0.876312,0.769770,0.218302,0.720637,0.136886,0.381739,0.143743,0.269741


DataFrame salvo em ../results/metrics_4_9
